In [1]:
!pip install --upgrade --quiet pip
print('pip update pip complete.')

pip update pip complete.


In [2]:
!pip install --quiet farm-haystack
print('pip install haystack complete.')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires pydantic>=2, but you have pydantic 1.10.14 which is incompatible.
pip install haystack complete.


In [3]:
URLS = [
    'http://www.textfiles.com/science/gems.01',
    'http://www.textfiles.com/science/gems.2',
    'http://www.textfiles.com/science/gems.3',
    'http://www.textfiles.com/science/gems.4',
]
print('we have {} URLs'.format(len(URLS)))

we have 4 URLs


In [4]:
# https://stackoverflow.com/a/56951135
import os
import requests


def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))

DESTINATION = '/kaggle/working/files/'

for url in URLS:
    download(url=url, dest_folder=DESTINATION)

saving to /kaggle/working/files/gems.01
saving to /kaggle/working/files/gems.2
saving to /kaggle/working/files/gems.3
saving to /kaggle/working/files/gems.4


In [5]:
import os
from glob import glob

for source_file in glob(DESTINATION + '*'):
    if not source_file.endswith('.txt'):
        os.rename(source_file, source_file + '.txt')

In [6]:
import arrow
from haystack.document_stores import InMemoryDocumentStore
from haystack.utils import add_example_data
from haystack.utils import build_pipeline

PROVIDER = ['anthropic', 'cohere', 'huggingface', 'openai'][2]
provider = PROVIDER

time_start = arrow.now()
document_store = InMemoryDocumentStore(use_bm25=True)
print('{} built document store'.format(arrow.now() - time_start))
add_example_data(document_store, '/kaggle/working/files')
print('{} added data from {}'.format(arrow.now() - time_start, DESTINATION))
pipeline = build_pipeline(provider=provider, API_BASE=None, API_KEY=None, document_store=document_store)
print('{} built pipeline'.format(arrow.now() - time_start))

2024-03-21 19:11:42.229094: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 19:11:42.229227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 19:11:42.390935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 19:12:00,297	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


0:00:00.000676 built document store


Updating BM25 representation...: 100%|██████████| 52/52 [00:00<00:00, 11161.34 docs/s]

0:00:00.129323 added data from /kaggle/working/files/


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

0:02:13.688119 built pipeline


In [7]:
import arrow

time_start = arrow.now()
query='What are gems?'

# result = pipeline.run(query=query)

print('done in {}'.format(arrow.now() - time_start))

done in 0:00:00.000144


In [8]:
from haystack.utils import print_answers

# what are the alternative details?
# print_answers(results=result, details='medium')